In [1]:
!pip install matplotlib librosa pandas scipy==1.3.0

ERROR: Error checking for conflicts.
Traceback (most recent call last):
  File "/usr/share/python-wheels/pkg_resources-0.0.0-py2.py3-none-any.whl/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/share/python-wheels/pkg_resources-0.0.0-py2.py3-none-any.whl/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/share/python-wheels/pkg_resources-0.0.0-py2.py3-none-any.whl/pkg_resources/__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "/usr/share/python-wheels/pkg_resources-0.0.0-py2.py3-none-any.whl/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _pkg_info

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/inf

In [1]:
import os
import IPython.display as ipd

import os
import numpy as np
import scipy
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import matplotlib.pyplot as plt
import pandas as pd  
import librosa as lb 
import librosa.display 

%matplotlib inline

# MFCCs

In [2]:
def extract_mfccs(audio_file_path):
    n_mfcc = 10
    y, sr = librosa.load(audio_file_path)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfccs.mean(axis=1) 

# Test Recording

In [3]:
import pyaudio
import wave

In [2]:
audio = pyaudio.PyAudio()

print("----------------------record device list---------------------")
info = audio.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
        if (audio.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
            print("Input Device id ", i, " - ", audio.get_device_info_by_host_api_device_index(0, i).get('name'))

print("-------------------------------------------------------------")


----------------------record device list---------------------
Input Device id  0  -  USB Device 0x46d:0x825: Audio (hw:0,0)
Input Device id  1  -  Microsoft Modern USB Headset: Audio (hw:1,0)
Input Device id  6  -  NVIDIA Jetson Orin NX APE: - (hw:3,0)
Input Device id  7  -  NVIDIA Jetson Orin NX APE: - (hw:3,1)
Input Device id  8  -  NVIDIA Jetson Orin NX APE: - (hw:3,2)
Input Device id  9  -  NVIDIA Jetson Orin NX APE: - (hw:3,3)
Input Device id  10  -  NVIDIA Jetson Orin NX APE: - (hw:3,4)
Input Device id  11  -  NVIDIA Jetson Orin NX APE: - (hw:3,5)
Input Device id  12  -  NVIDIA Jetson Orin NX APE: - (hw:3,6)
Input Device id  13  -  NVIDIA Jetson Orin NX APE: - (hw:3,7)
Input Device id  14  -  NVIDIA Jetson Orin NX APE: - (hw:3,8)
Input Device id  15  -  NVIDIA Jetson Orin NX APE: - (hw:3,9)
Input Device id  16  -  NVIDIA Jetson Orin NX APE: - (hw:3,10)
Input Device id  17  -  NVIDIA Jetson Orin NX APE: - (hw:3,11)
Input Device id  18  -  NVIDIA Jetson Orin NX APE: - (hw:3,12)
Inp

ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.USB-Audio.pcm.hdmi.0:CARD=0,AES0=4,AES1=130,AES2=0,AES3=2'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM hdmi
ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.USB-Audio.pcm.hdmi.0:CARD=0,AES0=4,AES1=130,AES2=0,AES3=2'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory


In [38]:
import pyaudio
import wave

chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 2
fs = 44100  # Record at 44100 samples per second
seconds = 10
filename = "output.wav"

p = pyaudio.PyAudio()  # Create an interface to PortAudio

print('Recording')

stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

frames = []  # Initialize array to store frames

# Store data in chunks for 3 seconds
for i in range(0, int(fs / chunk * seconds)):
    data = stream.read(chunk)
    frames.append(data)

# Stop and close the stream 
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('Finished recording')

# Save the recorded data as a WAV file
wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()

Recording
Finished recording


# Theoretical live data classification

In [4]:
import numpy as np
import pyaudio
import time
import librosa
import wave
import pickle

#for now this works but will need to add some filtering to get out the noise
class AudioHandler(object):
    def __init__(self, duration, svm, debug=False):
        self.FORMAT = pyaudio.paInt16
        self.CHANNELS = 2
        self.RATE = 8000
        self.CHUNK = 1024 * 2
        self.RECORD_SECONDS = 5
        self.DURATION = duration
        self.frame_to_proc = []
        self.audio_buffer = []
        self.n_mfcc = 10
        self.starttime = time.time()
        self.mfccs = None
        self.p = None
        self.stream = None


    def start(self):
        self.p = pyaudio.PyAudio()
        self.stream = self.p.open(format=self.FORMAT,
                                  channels=self.CHANNELS,
                                  rate=self.RATE,
                                  input=True,
                                  output=False,
                                  stream_callback=self.callback,
                                  frames_per_buffer=self.CHUNK)

    def stop(self, debug=False):
        self.audio_buffer.extend(self.frame_to_proc)
        audio_data = b''.join(self.frame_to_proc)
        audio_array = np.frombuffer(audio_data, dtype=np.int16)
        self.mfccs = librosa.feature.mfcc(y=audio_array.astype('float32'), 
                                          sr=self.RATE, n_mfcc=self.n_mfcc,dtype=np.float32).mean(axis=1)
        print('end mfccs',self.mfccs)
        self.frame_to_proc = []

        with wave.open('debug_out.wav', 'wb') as wf:
            wf.setnchannels(self.CHANNELS)
            wf.setsampwidth(self.p.get_sample_size(self.FORMAT))
            wf.setframerate(self.RATE)
            wf.writeframes(b''.join(self.audio_buffer))
        self.stream.close()
        self.p.terminate()

    #TODO - increase the buffer to 5 sec
    def callback(self, in_data, frame_count, time_info, flag):
        self.frame_to_proc.append(in_data)
        # print('processing')
        if len(self.frame_to_proc) * self.CHUNK  >= self.RECORD_SECONDS * self.RATE:
            self.audio_buffer.extend(self.frame_to_proc)
            audio_data = b''.join(self.frame_to_proc)
            audio_array = np.frombuffer(audio_data, dtype=np.int16)
            self.mfccs = librosa.feature.mfcc(y=audio_array.astype('float32'), 
                                              sr=self.RATE, n_mfcc=self.n_mfcc,dtype=np.float32).mean(axis=1)
            is_prediction = svm.predict(self.mfccs.reshape(1, -1))
            print('prediction',is_prediction,'clip mfccs',self.mfccs)
            self.frame_to_proc = []
        return None, pyaudio.paContinue

    def mainloop(self):
        while (self.stream.is_active() and (time.time() - self.starttime) < self.DURATION): # if using button you can set self.stream to 0 (self.stream = 0), otherwise you can use a stop condition
            time.sleep(1.0)


In [5]:
svm = pickle.load(open('svm_file', 'rb'))

In [7]:
audio = AudioHandler(10,svm)
audio.start()    # open the the stream
audio.mainloop()  
audio.stop()

ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.USB-Audio.pcm.hdmi.0:CARD=0,AES0=4,AES1=130,AES2=0,AES3=2'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM hdmi
ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.USB-Audio.pcm.hdmi.0:CARD=0,AES0=4,AES1=130,AES2=0,AES3=2'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directo

prediction ['positive'] clip mfccs [ 7.7184473e+02  5.1259922e+01 -3.4724632e+01 -4.9737705e+01
  6.0405165e-01 -3.7093327e+00 -3.8780872e+01  5.0108513e+01
 -1.7502642e+01  8.2919426e+00]
end mfccs [842.8826     53.74533   -31.407688  -63.427322   -2.7842946 -12.775025
 -41.97876    45.827106  -19.631779    3.6610267]


# Demo for wave file

In [31]:
import numpy as np
import pyaudio
import time
import librosa
import wave
import pickle

#for now this works but will need to add some filtering to get out the noise
class AudioHandler(object):
    def __init__(self, duration, svm, test_wav, debug=False):
        self.FORMAT = pyaudio.paInt16
        self.CHANNELS = 1
        self.RATE = 8000
        self.CHUNK = 1024 * 2
        self.RECORD_SECONDS = 5
        self.DURATION = duration
        self.frame_to_proc = []
        self.audio_buffer = []
        self.n_mfcc = 10
        self.starttime = time.time()
        self.mfccs = None
        self.p = None
        self.stream = None
        self.wf = wave.open(test_wav, 'rb')
        self.i = 0

    def start(self):
        self.p = pyaudio.PyAudio()
        self.FORMAT = self.p.get_format_from_width(self.wf.getsampwidth())
        print(self.FORMAT)
        self.CHANNELS = self.wf.getnchannels()
        self.RATE = self.wf.getframerate()
        print(self.CHANNELS, self.RATE)
        self.stream = self.p.open(format=self.FORMAT,
                                  channels=self.CHANNELS,
                                  rate=self.RATE,
                                  input=True,
                                  output=False,
                                  stream_callback=self.callback,
                                  frames_per_buffer=self.CHUNK)

    def stop(self, debug=False):
        self.audio_buffer.extend(self.frame_to_proc)
        audio_data = b''.join(self.frame_to_proc)
        audio_array = np.frombuffer(audio_data, dtype=np.int16)
        self.mfccs = librosa.feature.mfcc(y=audio_array.astype('float32'), 
                                          sr=self.RATE, n_mfcc=self.n_mfcc,dtype=np.float32).mean(axis=1)
        print('end mfccs',self.mfccs)
        self.frame_to_proc = []

        with wave.open('debug_out.wav', 'wb') as wf:
            wf.setnchannels(self.CHANNELS)
            wf.setsampwidth(self.p.get_sample_size(self.FORMAT))
            wf.setframerate(self.RATE)
            wf.writeframes(b''.join(self.audio_buffer))
        self.stream.close()
        self.p.terminate()

    #TODO - increase the buffer to 5 sec
    def callback(self, in_data, frame_count, time_info, flag):
        # self.frame_to_proc.append(in_data)
        self.frame_to_proc.append(self.wf.readframes(frame_count))
        # print('processing')
        if len(self.frame_to_proc) * self.CHUNK  >= self.RECORD_SECONDS * self.RATE:
            # print(self.frame_to_proc)
            self.audio_buffer.extend(self.frame_to_proc)
            audio_data = b''.join(self.frame_to_proc)
            audio_array = np.frombuffer(audio_data, dtype=np.int16)
            self.mfccs = librosa.feature.mfcc(y=audio_array.astype('float32'), 
                                              sr=8000, n_mfcc=10,dtype=np.float32).mean(axis=1)
            prediction = svm.predict(self.mfccs.reshape(1, -1))
            with wave.open(f"streamfile_{self.i}.wav", 'wb') as wf:
                wf.setnchannels(self.CHANNELS)
                wf.setsampwidth(self.p.get_sample_size(self.FORMAT))
                wf.setframerate(self.RATE)
                wf.writeframes(audio_data)
            self.i += 1
            print('prediction',prediction,'clip mfccs',self.mfccs)
            self.frame_to_proc = []
        return None, pyaudio.paContinue

    def mainloop(self):
        while (self.stream.is_active() and (time.time() - self.starttime) < self.DURATION): # if using button you can set self.stream to 0 (self.stream = 0), otherwise you can use a stop condition
            time.sleep(.1)

In [32]:
svm = pickle.load(open('svm_file', 'rb'))
knn = pickle.load(open('knn_cry_identifier', 'rb'))

In [33]:
audio = AudioHandler(30, svm, 'othersounds.wav')
audio.start()    # open the the stream
audio.mainloop()  
audio.stop()

ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.tegra-hda.pcm.front.0:CARD=0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM front
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib confmisc.c:1281:(snd_func_refer) Unable to find definition 'cards.tegra-hda.pcm.surround51.0:CARD=0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM surround21
ALSA lib confmisc.c:1281:(snd_func_refer) Unable to

8
1 8000
prediction ['positive'] clip mfccs [728.124      35.96248   -39.35407     7.3427615 -40.80658     0.8998822
 -30.285677   -6.675568  -14.886764   -1.7160983]
prediction ['positive'] clip mfccs [ 7.8139642e+02 -1.1181403e+01 -6.2541245e+01  8.3639393e+00
 -2.6040783e+01  7.8054380e+00 -3.0136505e+01 -4.5083186e-01
 -1.5260233e+01 -5.8595142e+00]
prediction ['positive'] clip mfccs [ 7.8730505e+02  7.4807558e+00 -4.3554279e+01 -4.9053850e+00
 -2.0938423e+01  5.7264752e+00 -2.4985460e+01  2.2682120e-01
 -1.5193882e+01 -8.6363974e+00]
prediction ['positive'] clip mfccs [820.3048     25.867136  -17.832552    9.936484  -26.937618    3.5067067
 -14.500987   -9.828624  -10.786496  -15.570577 ]


/home/infasafe/infasafe/.testvenv/lib/python3.8/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1920
  warnings.warn(


prediction ['negative'] clip mfccs [714.20996    78.82092    -6.9769654  25.186365  -10.576218   -5.8727064
 -14.664047  -11.588416  -13.87551   -14.107201 ]
end mfccs [-1131.3708     0.         0.         0.         0.         0.
     0.         0.         0.         0.    ]


/home/infasafe/infasafe/.testvenv/lib/python3.8/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(
